# Postprocessing of language_tool + run through evaluation
- for data_error_types
    - map to data_error_types
    - map to data_error_token
- for test run through (data_error_types_100)


- Error Types

In [2]:
import pandas as pd
import sys
sys.path.insert(0, '../../Lisa')
import litkey_2

In [3]:
# Do not truncate rows of DataFrame
pd.set_option('display.max_rows', None)
# Do not truncate column width
pd.set_option('display.max_colwidth', None)

## data_error_types_language_tool
- map to data_error_types
- map to data_error_token

In [5]:
data_error_types = pd.read_pickle('./pickles/data_error_types_language_tool_RULEID.pkl')
data_error_types.head(2)

,original,suggestions,rule_id
0,belt,"[Belt, Welt, bald, hält, Geld, Feld, Held, Zelt, bebt, bellt, perlt, pell, best, meld, peilt, pellt, pult, zelt, bei, seit]",GERMAN_SPELLER_RULE
1,kukt,"[rückt, zückt, bückt, guckt, kickt, kackt, gut, rückte, deckt, fügt, Lücke, lockt, weckt, packt, Mücke, kund, kürt, lügt, hackt, hockt]",GERMAN_SPELLER_RULE


In [6]:
# Get data_error_types original
df_orig = litkey_2.load(litkey_data_path = "../../litkey-data/")

print(df_orig.shape[0])
print(data_error_types.shape[0])

9484
8771


In [7]:
# Check if duplicate originals results in same suggestions
# By eye: Yes
dupl = data_error_types[data_error_types.duplicated(subset='original', keep=False)]
dupl.sort_values(by='original', ascending=True).head(10)

,original,suggestions,rule_id
422,Aaus,"[Aus, Haus, Baus, Maus, Raus, Gauß, Ahaus, Gaus, Taus, Kauz, Gäus, Aals, Laus, Aas, Saus, Anus, Asus, Als, Auf, Das]",GERMAN_SPELLER_RULE
3607,Aaus,"[Aus, Haus, Baus, Maus, Raus, Gauß, Ahaus, Gaus, Taus, Kauz, Gäus, Aals, Laus, Aas, Saus, Anus, Asus, Als, Auf, Das]",GERMAN_SPELLER_RULE
3847,Ales,"[Al es, Alex, Als, Alec, Alles, Ares, Wales, Aales, Altes, Alecs, Alfs, Sales, Alis, Tales, Alls, Yales, Fläz, Öles, Arles, Oles]",GERMAN_SPELLER_RULE
3345,Ales,"[Al es, Alex, Als, Alec, Alles, Ares, Wales, Aales, Altes, Alecs, Alfs, Sales, Alis, Tales, Alls, Yales, Fläz, Öles, Arles, Oles]",GERMAN_SPELLER_RULE
4674,Auser,"[Aus er, Außer, Auer, Äußer, Süßer, Außen, Büßer, Hauser, Mauser, Auster, Lauser, Augen, Äußere, Autor, Äußern, Häuser, Äußert, Guter, Äußre, Adler]",GERMAN_SPELLER_RULE
2553,Auser,"[Aus er, Außer, Auer, Äußer, Süßer, Außen, Büßer, Hauser, Mauser, Auster, Lauser, Augen, Äußere, Autor, Äußern, Häuser, Äußert, Guter, Äußre, Adler]",GERMAN_SPELLER_RULE
5160,Beld,"[Belt, Welt, Bald, Bild, Hält, Geld, Feld, Held, Bett, Zelt, Bebt, Pell, Beltz, Bälde, Belud, Beat, Best, Beet, Meld, Bert]",GERMAN_SPELLER_RULE
3687,Beld,"[Belt, Welt, Bald, Bild, Hält, Geld, Feld, Held, Bett, Zelt, Bebt, Pell, Beltz, Bälde, Belud, Beat, Best, Beet, Meld, Bert]",GERMAN_SPELLER_RULE
8537,Belen,"[Bären, Beten, Beleg, Selen, Besen, Sälen, Bällen, Bellen, Beben, Gelen, Beilen, Velen, Bäten, Peilen, Belsen, Pellen, Perlen, Beulen, Pesen, Beyen]",GERMAN_SPELLER_RULE
5428,Belen,"[Bären, Beten, Beleg, Selen, Besen, Sälen, Bällen, Bellen, Beben, Gelen, Beilen, Velen, Bäten, Peilen, Belsen, Pellen, Perlen, Beulen, Pesen, Beyen]",GERMAN_SPELLER_RULE


In [8]:
# Drop duplicates in df that went through language_tool (bc same original, same suggestions anyway)
# To be able to merge to original df

data_error_types.drop_duplicates(subset='original', keep='first', inplace=True)
data_error_types.shape[0] # before 8771, after 8481; Diff 290  

8481

In [9]:
# Merge
merged = df_orig.merge(data_error_types, how='left', on='original')
merged.shape[0]

9484

In [10]:
merged.fillna('', inplace=True)

In [11]:
# Evaluate suggestions
merged_eval = litkey_2.evaluate(merged)
merged_eval_cs = litkey_2.evaluate(merged, lower_case=False)

  0%|          | 0/9484 [00:00<?, ?it/s]

  0%|          | 0/9484 [00:00<?, ?it/s]

In [23]:
# Export
#merged_eval.to_pickle('data_error_types_language_tool_evaluation_lc.pkl')
#merged_eval_cs.to_pickle('data_error_types_language_tool_evaluation_cs.pkl')

In [12]:
# FOR TOKENS
df_token = litkey_2.load(litkey_data_path = "../../litkey-data/", toss_duplicates=False)

In [13]:
df_token.head(2)

,original,corrected,filename,freq_ori,freq_cor,freq_tup
0,belt,bellt,01-005-2-III-Eis.csv,91,138,91
1,kukt,kuckt,01-005-2-III-Eis.csv,73,152,73


In [14]:
merged_token = df_token.merge(data_error_types, how='left', on='original')
merged_token.fillna('', inplace=True)
merged_token.shape[0]

24601

In [15]:
# Evaluate
merged_token_eval_lc = litkey_2.evaluate(merged_token)
merged_token_eval_cs = litkey_2.evaluate(merged_token, lower_case=False)

  0%|          | 0/24601 [00:00<?, ?it/s]

  0%|          | 0/24601 [00:00<?, ?it/s]

In [33]:
# Export
#merged_token_eval_lc.to_pickle('data_error_token_language_tool_evaluation_lc.pkl')
#merged_token_eval_cs.to_pickle('data_error_token_language_tool_evaluation_cs.pkl')

In [19]:
#merged_token_eval_lc.to_pickle('./pickles/pickles_evaluation/data_error_token_language_tool_evaluation_lc_RULEID.pkl')

***
## data_error_types_100_language_tool

In [4]:
# Read in df
data_error_types = pd.read_pickle('data_error_types_100_language_tool.pkl')

In [5]:
display(data_error_types.head(10))
display(data_error_types.shape[0]) # 88

,original,suggestions
0,belt,[Belt]
1,kukt,"[rückt, zückt, bückt, guckt, kickt, kackt, gut, rückte, deckt, fügt, Lücke, lockt, weckt, packt, Mücke, kund, kürt, lügt, hackt, hockt]"
2,dan,"[dann, Dan, den, an, das, man, da, dar, van, San, tun, Jan, dank, tat, dran, ran, tank, Fan, Idan, Pan]"
3,gekricht,"[gebricht, gegeigt, gekickt, gekracht, gebracht, geprägt, gezeigt, Gericht, Bericht, gefragt, gerecht, gekocht, geneigt, gereicht, Rücksicht, gerückt, geklagt, geeicht, gegrillt, geknickt]"
4,leken,"[legen, locken, lägen, Säcken, lecken, gegen, wegen, leben, lagen, lesen, Becken, decken, dicken, legten, lenken, locker, regen, rücken, Lücken, Nacken]"
5,felt,"[Welt, fest, fällt, hält, Geld, fehlt, Feld, Held, fett, Zelt, fegt, Belt, feilt, feit, fielt, meld, pfählt, zelt, seit]"
6,wolte,"[wollte, holte, volle, wohnte, wolle, wilde, Nolte, Nolde, Bolte, Dolde, holde, wölbte, Golde, Solde, Volte, johlte, nölte, vote, walte, wogte]"
7,lekt,"[legt, lockt, leckt, bäckt, lägt, lebt, legte, deckt, lädt, rückt, lenkt, lockte, weckt, Bäcker, hegt, lege, leid, läge, packt, regt]"
8,fom,"[FOM, vom, von, vor, Rom, Dom, vorm, zoom, Boom, Tom, Zoom, im, dem, für, am, um, zum, so, ihm, kam]"
9,gawen,"[kamen, gaben, kämen, gäben, kauen, garen, gären, Laven, käsen, Maven, gegen, kann, waren, haben, Namen, davon, geben, gehen, bauen, ganzen]"


88

In [6]:
# Merge corrected back
df_orig = pd.read_csv('../Experiment_Boyd/Input/smaller_data_sets/2021-04-14_litkey_2_error_types_cs_NO_COUNT_100.tsv', sep="\t", header=None)
df_orig.rename(columns={0: 'original', 1:'corrected'}, inplace=True)
display(df_orig.shape[0]) # 100
display(df_orig.head(2))

100

,original,corrected
0,belt,bellt
1,kukt,kuckt


In [7]:
merged = df_orig.merge(data_error_types, how='left', on='original')

In [1]:
#merged.shape[0]
#merged

In [12]:
merged.fillna('', inplace=True)

In [13]:
# Evaluate suggestions
merged_eval = litkey_2.evaluate(merged)

  0%|          | 0/100 [00:00<?, ?it/s]

In [16]:
# First stats
merged_eval['sugg_idx'].fillna(-1, inplace=True)
merged_eval.sugg_idx.value_counts().sort_values(ascending=False)

 0.0     30
-1.0     28
 1.0     12
 2.0      9
 3.0      5
 4.0      5
 5.0      3
 6.0      3
 11.0     2
 9.0      1
 17.0     1
 19.0     1
Name: sugg_idx, dtype: int64

In [21]:
merged_eval.in_sugg.value_counts()

True     72
False    28
Name: in_sugg, dtype: int64

In [22]:
merged_eval.idx0.value_counts()

False    70
True     30
Name: idx0, dtype: int64

# Error Types

In [ ]:
# Extract again
# Text info
start_positions = []
end_positions = []
num_texts = []

my_mistakes = []
my_corrections = []

# Error info
ruleId = []
category = []
ruleIssueType = []


text_count = 0

for text in x:
    
    for match in text:
    
        start_positions.append(match.offset)
        end_positions.append(match.errorLength+match.offset)
        num_texts.append(text_count)
        
        my_mistakes.append(whole_texts[text_count][match.offset:match.errorLength+match.offset])
        my_corrections.append(match.replacements)
        
        ruleId.append(match.ruleId)
        category.append(match.category)
        ruleIssueType.append(match.ruleIssueType)
        
    text_count+=1    